In [3]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain import hub

In [7]:
%pip install pymupdf


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# 단계 1: 문서 로드(Load Documents)
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
file_path = "data/cleaned_doremi.txt"
loader = TextLoader(file_path=file_path, encoding="utf-8")


# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

split_docs = loader.load_and_split(text_splitter=text_splitter)

# 단계 3: 임베딩 & 벡터스토어 생성(Create Vectorstore)
# 벡터스토어를 생성합니다.
# vectorstore = FAISS.from_documents(documents=split_docs, embedding=OpenAIEmbeddings())

embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs[:50], embedding=embedding)  # 먼저 50개로 시작

# 나머지는 add_documents로 추가
batch_size = 50
for i in range(50, len(split_docs), batch_size):
    batch = split_docs[i:i+batch_size]
    vectorstore.add_documents(batch)

# Step 4: 저장
vectorstore.save_local("db")
print("✅ 벡터스토어 저장 완료!")

# 단계 4: 검색(Search)
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

# 단계 5: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = hub.pull("rlm/rag-prompt")

# 단계 6: 언어모델 생성(Create LLM)
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)


# 단계 7: 체인 생성(Create Chain)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 단계 8: 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "김첨지의 부인은 무엇이 먹고 싶다고 했나요?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"PDF Path: {file_path}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

✅ 벡터스토어 저장 완료!
PDF Path: data/cleaned_doremi.txt
[HUMAN]
김첨지의 부인은 무엇이 먹고 싶다고 했나요?

[AI]
I'm sorry, but the provided context does not contain information about what 김첨지's wife wanted to eat.


In [14]:
question = "정원은 희원과 왜 멀어졌나요?"
response = rag_chain.invoke(question)

# 결과 출력
print(f"Path: {file_path}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

Path: data/cleaned_doremi.txt
[HUMAN]
정원은 희원과 왜 멀어졌나요?

[AI]
정원은 희원을 믿지 않고 시험했기 때문에 희원과 멀어졌습니다. 이로 인해 희원이 걱정하게 되었고, 정원은 오히려 희원을 원망하게 되었습니다. 이러한 오해와 불신이 그들 사이를 멀어지게 만든 원인입니다.
